In [ ]:
!pip install fastapi uvicorn transformers torch torchvision ultralytics uvicorn pyngrok yolov8 langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/final_model')

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langdetect import detect, DetectorFactory
from torchvision import models, transforms
from PIL import Image
from ultralytics import YOLO
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import requests
from io import BytesIO
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("CombinedApp")

# Initialize the FastAPI app
app = FastAPI()

# Paths to models
distilbert_path = "/content/drive/MyDrive/final_model/finetuned_DBT"
mobilenet_path = "/content/drive/MyDrive/final_model/NSFWMN_FINAL.pth"
yolov8_path = "/content/drive/MyDrive/final_model/FINALTUNED.pt"

# Load DistilBERT
try:
    distilbert_tokenizer = DistilBertTokenizer.from_pretrained(distilbert_path)
    distilbert_model = DistilBertForSequenceClassification.from_pretrained(distilbert_path)
    distilbert_model.eval()
    logger.info("DistilBERT model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading DistilBERT model: {e}")
    raise RuntimeError(f"Failed to load DistilBERT: {e}")

# Load MobileNetV2
try:
    mobilenet_model = models.mobilenet_v2(pretrained=False)
    mobilenet_model.classifier[1] = torch.nn.Linear(mobilenet_model.last_channel, 2)
    mobilenet_model.load_state_dict(torch.load(mobilenet_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
    mobilenet_model.eval()
    logger.info("MobileNetV2 model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading MobileNetV2 model: {e}")
    raise RuntimeError(f"Failed to load MobileNetV2: {e}")

# Load YOLOv8
try:
    yolo_model = YOLO(yolov8_path)
    logger.info("YOLOv8 model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading YOLOv8 model: {e}")
    raise RuntimeError(f"Failed to load YOLOv8: {e}")

# Image preprocessing pipeline for MobileNetV2
mobilenet_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Input validation models
class TextInput(BaseModel):
    text: str

class ImageInput(BaseModel):
    image_url: str

# Fetch and preprocess image
def fetch_image_from_url(url):
    try:
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)).convert("RGB")
    except requests.exceptions.RequestException as e:
        raise HTTPException(status_code=400, detail=f"Error fetching image: {e}")

# Process text with DistilBERT
def process_text(text):
    try:
        # Detect language using langdetect
        language = detect(text)
        if language != "en":
            return "NON-ENGLISH"  # Skip non-English text

        # Process English text with DistilBERT
        inputs = distilbert_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = distilbert_model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        _, prediction = torch.max(probabilities, dim=-1)
        label = "NEGATIVE" if prediction.item() == 0 else "POSITIVE"
        return label
    except Exception as e:
        logger.error(f"Error processing text: {e}")
        raise HTTPException(status_code=500, detail="Error processing text input.")

# Process image with MobileNetV2
def process_image(image):
    try:
        input_tensor = mobilenet_transforms(image).unsqueeze(0)
        outputs = mobilenet_model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, prediction = torch.max(probabilities, dim=-1)
        label = "NEGATIVE" if prediction.item() == 0 else "POSITIVE"
        return label
    except Exception as e:
        logger.error(f"Error processing image: {e}")
        raise HTTPException(status_code=500, detail="Error processing image input.")

# Perform object detection with YOLOv8
def detect_objects_with_confidence(image):
    try:
        results = yolo_model.predict(image, save=False)
        yolo_confidence = 0.0
        yolo_label = "POSITIVE"  # Default label
        if results and results[0].boxes is not None:
            for box in results[0].boxes:
                cls_conf = float(box.conf.cpu().numpy())
                yolo_confidence = max(yolo_confidence, cls_conf)  # Use the highest confidence box
                # Example logic: classify as NEGATIVE if any object is detected with high confidence
                if yolo_confidence > 0.5:
                    yolo_label = "NEGATIVE"
        return yolo_label, yolo_confidence
    except Exception as e:
        logger.error(f"Error during YOLOv8 classification: {e}")
        raise HTTPException(status_code=500, detail="Error in YOLOv8 classification.")

# Ensemble logic for image classification
def ensemble_image_classification(image):
    """
    Combine MobileNetV2 and YOLOv8 outputs into a unified decision.
    """
    mobilenet_label, mobilenet_conf = process_image(image)
    yolo_label, yolo_conf = detect_objects_with_confidence(image)

    # Combine results based on confidence
    if mobilenet_conf >= yolo_conf:
        return mobilenet_label
    else:
        return yolo_label


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Allowed origins for CORS
allowed_origins = [
    "chrome-extension://pmojffjdjboijhgjifjdhhkkgfpcodmi",  # Chrome extension
    "https://peeredfast.ngrok.app",  # Ngrok domain
    "https://x.com",               # Twitter
    "https://www.facebook.com",    # Facebook
]

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=allowed_origins,
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)

# Text prediction endpoint
@app.post("/text/")
def predict_text(payload: TextInput):
    """
    Endpoint for processing text using DistilBERT.
    """
    if not payload.text.strip():  # Check for empty or whitespace-only input
        raise HTTPException(status_code=400, detail="Text input cannot be empty.")
    try:
        # Process the text and classify it
        text_label = process_text(payload.text)

        # Return the result with the original text
        return {
            "text_label": text_label,
            "text": payload.text  # Include the original text
        }
    except Exception as e:
        logger.error(f"Error in text prediction endpoint: {e}")
        raise HTTPException(status_code=500, detail="Error in text prediction.")

# Image prediction endpoint
@app.post("/image/")
def predict_image(payload: ImageInput):
    """
    Endpoint for processing images using MobileNetV2 and YOLOv8.
    """
    if not payload.image_url.strip():  # Validate non-empty URL
        raise HTTPException(status_code=400, detail="The 'image_url' field cannot be empty.")
    try:
        # Fetch and preprocess image
        image = fetch_image_from_url(payload.image_url)

        # Image classification with MobileNetV2
        image_label = process_image(image)

        # Return the result with the original image URL
        return {
            "image_label": image_label,
            "image_url": payload.image_url
        }
    except Exception as e:
        logger.error(f"Error in image prediction endpoint: {e}")
        raise HTTPException(status_code=500, detail="Error in image prediction.")


In [ ]:
import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Ngrok Authentication
ngrok.set_auth_token("2pcZjzF1wHEVwZqhVooT1WlC1de_45tX2VJdSwhbbpCboCiYC")

# Start the Ngrok tunnel
public_url = ngrok.connect(8000, proto="http", hostname="peeredfast.ngrok.app")  # Use your custom domain
print(f"Public URL: {public_url}")

# Verify if the custom domain is correctly set
if str(public_url).startswith("http://peeredfast.ngrok.app") or str(public_url).startswith("https://peeredfast.ngrok.app"):
    print(f"Your custom domain {public_url} is working correctly!")
else:
    print(f"Expected URL mismatch: Got {public_url}, expected peeredfast.ngrok.app")

# Run the FastAPI server using uvicorn.Config to avoid conflict with nest_asyncio
# We create a config and pass the loop='none' arg to prevent starting a new event loop
config = uvicorn.Config(app, host="0.0.0.0", port=8000, loop="none") # Create a Uvicorn config object, explicitly setting the loop to "none"
server = uvicorn.Server(config)  # Create a Uvicorn server instance using the config

# Use nest_asyncio to run the server within the current event loop
nest_asyncio.apply() # Apply nest_asyncio *after* the uvicorn server is set up
server.run()

Public URL: NgrokTunnel: "https://peeredfast.ngrok.app" -> "http://localhost:8000"
Expected URL mismatch: Got NgrokTunnel: "https://peeredfast.ngrok.app" -> "http://localhost:8000", expected peeredfast.ngrok.app


INFO:     Started server process [578]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     45.114.134.186:0 - "OPTIONS /image/ HTTP/1.1" 200 OK
INFO:     45.114.134.186:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     45.114.134.186:0 - "OPTIONS /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "OPTIONS /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "OPTIONS /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     45.114.134.186:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     45.114.134.186:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP

ERROR:CombinedApp:Error processing text: No features in text.
ERROR:CombinedApp:Error in text prediction endpoint: 500: Error processing text input.


INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 500 Internal Server Error
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /text/ HTTP/1.1" 200 OK
INFO:     103.16.168.34:0 - "POST /image/ H

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [578]


KeyboardInterrupt: 